butuh bantuan scroling secara manual untuk mengambil data riviews google map dengan opitmal

In [ ]:
# butuh bantuan scroling secara manual untuk mengambil data riviews google map dengan opitmal

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, pandas as pd

options = Options()
options.add_argument('--lang=id')
options.add_argument('accept-language=id-ID,id')
driver = webdriver.Chrome(options=options)

url = "https://www.google.com/maps/place/McDonald's+-+Plasa+Marina/@-7.3163317,112.7378479,15z/data=!4m8!3m7!1s0x2dd7fb1e4fba2b2d:0xfd066b59ea41ee4f!8m2!3d-7.3163531!4d112.7481476!9m1!1b1!16s%2Fg%2F11hyt5mz4r?entry=ttu&g_ep=EgoyMDI1MDkxNy4wIKXMDSoASAFQAw%3D%3D"

driver.get(url)
time.sleep(5)

# Ambil nama tempat
try:
    nama_tempat = driver.find_element(By.CLASS_NAME, 'DUw0vf').text.strip()
except:
    nama_tempat = 'Unknown'

# Klik "Lainnya" agar semua review tampil
try:
    driver.find_element(
        By.XPATH,
        '//span[contains(text(), "Lainnya") or contains(text(), "More reviews")]/ancestor::button'
    ).click()
    time.sleep(3)
except:
    pass

reviews, seen = [], set()
max_reviews = 7000  # maksimal data yang akan diambil

while len(reviews) < max_reviews:
    cards = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf')
    if not cards:
        break

    for c in cards:
        try:
            user_el = c.find_elements(By.CLASS_NAME, 'd4r55')
            review_el = c.find_elements(By.CLASS_NAME, 'wiI7pd')
            rating_el = c.find_elements(By.CLASS_NAME, 'kvMYJc')

            user = user_el[0].text if user_el else 'Unknown'
            review = review_el[0].text.strip() if review_el else ''  # kosong tetap disimpan
            rating = rating_el[0].get_attribute('aria-label').split()[0] if rating_el else 'Unknown'

            if (user, review) not in seen:
                seen.add((user, review))
                reviews.append({
                    'nama_tempat': nama_tempat,
                    'user': user,
                    'review': review,
                    'rating': rating
                })
                print(f"Total data: {len(reviews)}")

        except Exception as e:
            print("Error parsing card:", e)
            continue

    # Scroll untuk load review baru
    try:
        driver.execute_script("arguments[0].scrollIntoView();", cards[-1])
        driver.execute_script("window.scrollBy(0, 200);")  # dorong lebih jauh
        time.sleep(2)
    except:
        break

# Convert ke DataFrame
df = pd.DataFrame(reviews)
print(df.head())

# Simpan ke CSV
df.to_csv("reviews_next.csv", index=False, encoding="utf-8-sig")
print(f"Total review tersimpan: {len(df)}")

driver.quit()


In [29]:
# di jalankan ketika keyboard interrupt

# Convert ke DataFrame
df = pd.DataFrame(reviews)
print(df.head())

# Simpan ke CSV
df.to_csv("reviews3.csv", index=False, encoding="utf-8-sig")
print(f"Total review tersimpan: {len(df)}")

driver.quit()


  nama_tempat               user  \
0     Unknown     Farah Seftiana   
1     Unknown  Mayong Mayangsari   
2     Unknown          Suci Axel   
3     Unknown     Darryl Lumeisa   
4     Unknown       melinda hadi   

                                              review rating  
0  Salah satu mcd yang ada di mall selain yang di...      5  
1  Tempatnya nyaman, luas dan bersih. Tersedia te...      5  
2  Saya pesen makan yg haruse dianter tapi gk dia...      1  
3  Ayamnya gosong , rugi beli mahal , sambal sc t...      1  
4  Pesan drive thrue tgl 14 september'25  pk.21.3...      1  
Total review tersimpan: 2146
